In [1]:
import numpy as np

In [122]:
tests = [
    {'input':[[0,1,2,3,4,5,6,7,8,9], 7], 'output':7},
    {'input':[[2,4,6,8,10,12,14], 10], 'output':4},
    {'input':[[0,1,2,6,7,8,9], 9], 'output':6},
    {'input':[[6,8,10,12,14], 6], 'output':0},
    {'input':[[6,8,10,12,14], 60], 'output':-1},
    {'input':[[2,4,6,8,10,12,14], 9], 'output':-1}
]

In [123]:
def search_num(num_list, num_to_find):
    high_band = len(num_list) - 1
    low_band = 0
    result = high_band // 2
    mid = num_list[result]
    i = 0

    if num_list[high_band] == num_to_find:
        result = high_band

    while mid != num_to_find and i < 12:
        i += 1

        if mid > num_to_find:
            high_band = result
        
        elif mid < num_to_find:
            low_band = result
        
        if (high_band - low_band) // 2 == 0:
            if num_list[low_band] != num_to_find:
                result = -1
                break

        result = (high_band - low_band) // 2 + low_band
        mid = num_list[result]
    
    return result

In [124]:
for test in tests:
    print(search_num(*test['input']), search_num(*test['input']) == test['output'])

7 True
4 True
6 True
0 True
-1 True
-1 True
5 False
